In [257]:
from nltk import word_tokenize
from collections import defaultdict
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
import io
import numpy as np
import nltk

In [202]:
df = pd.read_csv('o365Data.csv', dtype=object)

In [204]:
all_subjects = df['EmailSubject'].tolist()

In [205]:
receipt_subjects = [s.lower() for s in all_subjects if isinstance(s,str)]

In [210]:
import mailbox

In [211]:
mbox = mailbox.mbox('emails.mbox')

In [246]:
non_receipt_emails = []
non_receipt_subjects = []
for message in mbox:
    payload = message.get_payload(decode=False)
    non_receipt_subjects.append(message['Subject'])
    if isinstance(payload,list):
        non_receipt_emails.append(payload[0])
    else:
        non_receipt_emails.append(payload) 


In [311]:
def clean_gmail_subjects(subs): 
    subs = [s for s in subs if isinstance(s,str)]
    
    remove_words = ['invoice','utf','utf-8','receipt','amazon','paypal','payment','confirmation','hughes','trip','booking','lance','weston','order','reservation','confirmed','itinerary','statement','folio','bill']

    return[s.lower() for s in subs if not any(word.lower() in s.lower() for word in remove_words)]

In [312]:
non_receipt_subjects = clean_gmail_subjects(non_receipt_subjects)

In [313]:
import random

def get_test_data_labels(zeros_list,ones_list):
    labels = np.zeros(len(zeros_list)).tolist() + np.ones(len(ones_list)).tolist()
    
    all_data = zeros_list + ones_list
    combined = list(zip(all_data, labels))
    random.shuffle(combined)

    all_data[:], labels[:] = zip(*combined)
    return all_data,labels

In [314]:
subjects, labels = get_test_data_labels(non_receipt_subjects,receipt_subjects)

In [318]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
X_test = []
y_test = []
predicted = []

def classify(data, labels):
    global X_test,y_test,predicted
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)
    vectorizer = CountVectorizer()
#     clf = MultinomialNB()
    clf = SGDClassifier(loss='hinge', penalty='l1', alpha=1e-4, random_state=42, max_iter=7, tol=None)
    pipe = Pipeline([('vect', vectorizer),
                        ('tfidf', TfidfTransformer()),
                        ('clf', clf)
                        ])
                        
    pipe.fit(X_train, y_train)  
    predicted = pipe.predict(X_test)
    
    return np.mean(predicted == y_test), clf, vectorizer

In [319]:
accuracy, clf, vectorizer = classify(subjects,labels)
print(accuracy)
print(type(classifier))

0.9750305182554655
<class 'sklearn.pipeline.Pipeline'>


In [320]:
z = [X_test[i] for i in range(len(X_test)) if y_test[i] == 1 and predicted[i] == 0]
z



['fwd: concur alert: outstanding credit card transactions - 20 days for neha saxena: need approval : please help. ',
 'your subscription renewal: kiosk browser remote subscription 30 days [and180820-1759-61163b]',
 'drivenow ebilling 9356158079',
 'chattanooga',
 'fw: your  autopay is in 5 days',
 'caitlin gardner(balnoca1@memorialhealth.com) sent you an email from www.marriott.com',
 'login to concur',
 'your subscription is updated',
 'ext: 24 hour reminder for pickup on sunday, september 16, 2018',
 'fw: simplepay - sikeres fizetés - http://fotaxi.hu ',
 'jacks',
 'ihre zahlung an taxi dolunay yildirimci',
 'july internet',
 'thanks for your purchase',
 'mauch, michael conf#782628989',
 'vickery cafe catering for 03/07/17',
 're: topgolf orlando - topgolf orlando event',
 'dellemc coffee 7.12.18',
 'login to concur',
 'recibo carlson wagonlit travel - juliano fantinatti',
 '5.30.18 bcr vip after party',
 '2018 cns annual meeting housing refund request for hill',
 'new: updates to th

In [221]:
body = "here is your uber ride"


classifier.predict([body])


array([1.])

In [222]:
def show_most_informative_features(vectorizer, clf, n=20):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print ("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

In [256]:
show_most_informative_features(vectorizer,clf,1000)

	-5.7231	lance          		14.7808	receipt        
	-4.4208	talk           		10.7101	payment        
	-4.1064	xbox           		9.4909	invoice        
	-4.0418	internships    		8.9160	confirmation   
	-2.9863	weekly         		7.9987	order          
	-2.9299	apps           		7.2226	reservation    
	-2.8230	game           		6.8776	ride           
	-2.7333	concurlabs     		4.9843	folio          
	-2.6029	now            		4.6683	booking        
	-2.5887	ohio           		4.5849	de             
	-2.5573	anniversary    		4.4609	itinerary      
	-2.4219	rate           		4.4468	ref            
	-2.4118	new            		4.2843	amazon         
	-2.2735	in             		4.1437	trip           
	-2.1690	off            		4.0670	outlook        
	-2.0577	haul           		3.5779	national       
	-1.9114	fiesta         		3.5578	enterprise     
	-1.9114	pumpkin        		3.4261	stay           
	-1.8696	deposit        		3.3866	charges        
	-1.8312	news           		3.3547	9061395446     
	-1.8166	hughes   

In [38]:
import eli5
eli5.show_weights(clf, top=10)